In [8]:
!pip install chromadb==0.5.15

  Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl.metadata (541 bytes)
Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl (413 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.3
    Uninstalling protobuf-5.28.3:
      Successfully uninstalled protobuf-5.28.3


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.64.1 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.5 which is incompatible.


In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

df = pd.read_csv(r"D:\2024_bigcontest\data\final_data\JEJU_MCT_KEYWORD_v2.csv")
df.head(4)

,가게명,주소,업종,이용건수구간,이용금액구간,건당평균이용금액구간,월_이용건수비중,화_이용건수비중,수_이용건수비중,목_이용건수비중,...,혼자,아이,부모님,반려동물,친척・형제,기타,주변관광지,mct_요약,keyword_요약,요약
0,비스트로낭,제주 서귀포시 안덕면 감산리 95-1번지 1층,이탈리아음식,4_50~75%,3_25~50%,2_10~25%,0.227273,0.090909,0.068182,0.181818,...,0.008772,0.166667,0.175439,0.0,0.043860,0.0,건강과성박물관,"비스트로낭의 업종은 양식, 위치는 제주 서귀포시 안덕면 감산리 95-1번지 1층 입...","비스트로낭의 가게유형은 이탈리아음식, 위치는 제주 서귀포시 안덕면 감산리 95-1번...","비스트로낭의 업종은 양식, 위치는 제주 서귀포시 안덕면 감산리 95-1번지 1층 입..."
1,예원이네,제주 서귀포시 안덕면 감산리 359-1번지 1층,"해물,생선요리",2_10~25%,1_상위 10% 이하,3_25~50%,0.197628,0.130435,0.000000,0.173913,...,0.000000,0.198473,0.221374,0.0,0.068702,0.0,건강과성박물관,"예원이네의 업종은 가정식, 위치는 제주 서귀포시 안덕면 감산리 359-1번지 1층 ...","예원이네의 가게유형은 해물,생선요리, 위치는 제주 서귀포시 안덕면 감산리 359-1...","예원이네의 업종은 가정식, 위치는 제주 서귀포시 안덕면 감산리 359-1번지 1층 ..."
2,기시어멍김밥,제주 서귀포시 강정동 232번지 1층 102호,분식,2_10~25%,3_25~50%,6_90% 초과(하위 10% 이하),0.168950,0.000000,0.146119,0.146119,...,0.202381,0.261905,0.023810,0.0,0.047619,0.0,"빠레브 호텔 제주, 호텔 토스카나, 제주 웨스턴 그레이스 호텔, 켄싱턴리조트 서귀포","기시어멍김밥의 업종은 분식, 위치는 제주 서귀포시 강정동 232번지 1층 102호 ...","기시어멍김밥의 가게유형은 분식, 위치는 제주 서귀포시 강정동 232번지 1층 102...","기시어멍김밥의 업종은 분식, 위치는 제주 서귀포시 강정동 232번지 1층 102호 ..."
3,귤밭사이로수제돈가스,제주 서귀포시 강정동 587-1번지 1층,돈가스,4_50~75%,5_75~90%,5_75~90%,0.140625,0.125000,0.250000,0.156250,...,0.102564,0.179487,0.025641,0.0,0.025641,0.0,"빠레브 호텔 제주, 호텔 토스카나, 제주 웨스턴 그레이스 호텔, 켄싱턴리조트 서귀포","귤밭사이로수제돈가스의 업종은 양식, 위치는 제주 서귀포시 강정동 587-1번지 1층...","귤밭사이로수제돈가스의 가게유형은 돈가스, 위치는 제주 서귀포시 강정동 587-1번지...","귤밭사이로수제돈가스의 업종은 양식, 위치는 제주 서귀포시 강정동 587-1번지 1층..."


In [2]:
df.columns

Index(['가게명', '주소', '업종', '이용건수구간', '이용금액구간', '건당평균이용금액구간', '월_이용건수비중',
       '화_이용건수비중', '수_이용건수비중', '목_이용건수비중', '금_이용건수비중', '토_이용건수비중', '일_이용건수비중',
       '05-11시_이용건수비중', '12-13시_이용건수비중', '14-17시_이용건수비중', '18-22시_이용건수비중',
       '23-04시_이용건수비중', '현지인_이용건수비중', '남성_이용건수비중', '여성_이용건수비중', '10-20대_회원비중',
       '30대_회원비중', '40대_회원비중', '50대_회원비중', '60대이상_회원비중', '상위키워드', '예약_무',
       '예약_유', '포장배달_유무', '바로 입장', '10분 이내', '30분 이내', '30분 이상', '1시간 이상',
       '2시간 이상', '데이트', '친목', '나들이', '여행', '일상', '기념일', '회식', '가족모임', '비즈니스',
       '연인・배우자', '친구', '지인・동료', '혼자', '아이', '부모님', '반려동물', '친척・형제', '기타',
       '주변관광지', 'mct_요약', 'keyword_요약', '요약'],
      dtype='object')

In [3]:
df['현지인'] = np.where(df['현지인_이용건수비중'] >= 0.48, 1, 0)
df['현지인']

0       0
1       0
2       1
3       1
4       1
       ..
4186    0
4187    0
4188    0
4189    0
4190    1
Name: 현지인, Length: 4191, dtype: int32

In [4]:
df[df['현지인'] == 1]

,가게명,주소,업종,이용건수구간,이용금액구간,건당평균이용금액구간,월_이용건수비중,화_이용건수비중,수_이용건수비중,목_이용건수비중,...,아이,부모님,반려동물,친척・형제,기타,주변관광지,mct_요약,keyword_요약,요약,현지인
2,기시어멍김밥,제주 서귀포시 강정동 232번지 1층 102호,분식,2_10~25%,3_25~50%,6_90% 초과(하위 10% 이하),0.168950,0.000000,0.146119,0.146119,...,0.261905,0.023810,0.000000,0.047619,0.000000,"빠레브 호텔 제주, 호텔 토스카나, 제주 웨스턴 그레이스 호텔, 켄싱턴리조트 서귀포","기시어멍김밥의 업종은 분식, 위치는 제주 서귀포시 강정동 232번지 1층 102호 ...","기시어멍김밥의 가게유형은 분식, 위치는 제주 서귀포시 강정동 232번지 1층 102...","기시어멍김밥의 업종은 분식, 위치는 제주 서귀포시 강정동 232번지 1층 102호 ...",1
3,귤밭사이로수제돈가스,제주 서귀포시 강정동 587-1번지 1층,돈가스,4_50~75%,5_75~90%,5_75~90%,0.140625,0.125000,0.250000,0.156250,...,0.179487,0.025641,0.000000,0.025641,0.000000,"빠레브 호텔 제주, 호텔 토스카나, 제주 웨스턴 그레이스 호텔, 켄싱턴리조트 서귀포","귤밭사이로수제돈가스의 업종은 양식, 위치는 제주 서귀포시 강정동 587-1번지 1층...","귤밭사이로수제돈가스의 가게유형은 돈가스, 위치는 제주 서귀포시 강정동 587-1번지...","귤밭사이로수제돈가스의 업종은 양식, 위치는 제주 서귀포시 강정동 587-1번지 1층...",1
4,광명대창집신서귀포점,제주 서귀포시 강정동 207-2번지 1층,"곱창,막창,양",6_90% 초과(하위 10% 이하),5_75~90%,2_10~25%,0.176471,0.176471,0.058824,0.000000,...,0.000000,0.000000,0.142857,0.000000,0.000000,"빠레브 호텔 제주, 호텔 토스카나, 제주 웨스턴 그레이스 호텔, 켄싱턴리조트 서귀포","광명대창집신서귀포점의 업종은 단품요리 전문, 위치는 제주 서귀포시 강정동 207-2...","광명대창집신서귀포점의 가게유형은 곱창,막창,양, 위치는 제주 서귀포시 강정동 207...","광명대창집신서귀포점의 업종은 단품요리 전문, 위치는 제주 서귀포시 강정동 207-2...",1
5,교촌치킨 서귀포혁신도시점,제주 서귀포시 강정동 166-10번지 1층,"치킨,닭강정",4_50~75%,4_50~75%,4_50~75%,0.064516,0.096774,0.112903,0.193548,...,0.368421,0.105263,0.000000,0.000000,0.000000,"빠레브 호텔 제주, 호텔 토스카나, 제주 웨스턴 그레이스 호텔, 켄싱턴리조트 서귀포","교촌치킨 서귀포혁신도시점의 업종은 치킨, 위치는 제주 서귀포시 강정동 166-10번...","교촌치킨 서귀포혁신도시점의 가게유형은 치킨,닭강정, 위치는 제주 서귀포시 강정동 1...","교촌치킨 서귀포혁신도시점의 업종은 치킨, 위치는 제주 서귀포시 강정동 166-10번...",1
7,고봉민김밥인(서귀포강정점),제주 서귀포시 강정동 208-1번지 1층 102호,분식,2_10~25%,5_75~90%,6_90% 초과(하위 10% 이하),0.156250,0.000000,0.137500,0.212500,...,0.500000,0.000000,0.000000,0.000000,0.000000,"빠레브 호텔 제주, 호텔 토스카나, 제주 웨스턴 그레이스 호텔, 켄싱턴리조트 서귀포","고봉민김밥인(서귀포강정점)의 업종은 분식, 위치는 제주 서귀포시 강정동 208-1번...","고봉민김밥인(서귀포강정점)의 가게유형은 분식, 위치는 제주 서귀포시 강정동 208-...","고봉민김밥인(서귀포강정점)의 업종은 분식, 위치는 제주 서귀포시 강정동 208-1번...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4179,효돈식(효돈식),제주 서귀포시 하효동 330-1번지 1층,한식,4_50~75%,5_75~90%,4_50~75%,0.095238,0.214286,0.119048,0.166667,...,0.146341,0.048780,0.000000,0.024390,0.024390,NaN,"효돈식(효돈식)의 업종은 가정식, 위치는 제주 서귀포시 하효동 330-1번지 1층 ...","효돈식(효돈식)의 가게유형은 한식, 위치는 제주 서귀포시 하효동 330-1번지 1층...","효돈식(효돈식)의 업종은 가정식, 위치는 제주 서귀포시 하효동 330-1번지 1층 ...",1
4181,효자동솥뚜껑 제주한림협재점,제주 제주시 한림읍 한림리 885-16번지 1층,돼지고기구이,4_50~75%,4_50~75%,4_50~75%,0.181818,0.145455,0.127273,0.163636,...,0.268908,0.092437,0.000000,0.058824,0.008403,NaN,"효자동솥뚜껑 제주한림협재점의 업종은 가정식, 위치는 제주 제주시 한림읍 한림리 88...","효자동솥뚜껑 제주한림협재점의 가게유형은 돼지고기구이, 위치는 제주 제주시 한림읍 한...","효자동솥뚜껑 제주한림협재점의 업종은 가정식, 위치는 제주 제주시 한림읍 한림리 88...",1
4182,후년하우스,제주 제주시 삼양이동 2181-19번지,샤브샤브,5_75~90%,4_50~75%,3_25~50%,0.074074,0.148148,0.148148,0.074074,...,0.181818,0.090909,0.000000,0.000000,0.000000,NaN,"후년하우스의 업종은 가정식, 위치는 제주 제주시 삼양이동 2181-19번지 입니다....","후년하우스의 가게유형은 샤브샤브, 위치는 제주 제주시 삼양이동 2181-19번지 입...","후년하우스의 업종은 가정식, 위치는 제주 제주시 삼양이동 2181-19번지 입니다....",1
4184,훔쳐온뒷고기,제주 제주시 도남동 918-9번지,돼지고기구이,6_90% 초과(하위 10% 이하),5_75~90%,2_10~25%,0.000000,0.176471,0.235294,0.176471,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,"훔쳐온뒷고기의 업종은 단품요리 전문, 위치는 제주 제주시 도남동 918-9번지 입니...","훔쳐온뒷고기의 가게유형은 돼지고기구이, 위치는 제주 제주시 도남동 918-9번지 입...","훔쳐온뒷고기의 업종은 단품요리 전문, 위치는 제주 제주시 도남동 918-9번지 입니...",1


In [5]:
df['요약'][0]

'비스트로낭의 업종은 양식, 위치는 제주 서귀포시 안덕면 감산리 95-1번지 1층 입니다.\n주요 키워드는 파스타, 뇨끼, 분위기, 메뉴, 스테이크, 소스, 식사, 피자, 문어, 재료, 예약, 모두, 아이, 사장, 직원, 설명, 여기, 엄마, 샐러드, 양식 입니다.\n주변 관광지는 건강과성박물관 입니다.\n이용건수는 동일 업종 내 상위 50~75%이고, 현지인 이용건수 비중은 0.25로, 순위는 전체 가게 중 2588위 입니다.\n이용건수가 가장 많은 요일은 월요일(0.23), 전체 가게 중 81위 입니다. 두 번째로 많은 요일은 일요일(0.20), 전체 가게 중 204위 입니다.\n이용건수가 가장 많은 시간대는 18-22시(0.43), 전체 가게 중 779위 입니다. 두 번째로 많은 시간대는 12-13시(0.34), 전체 가게 중 612위 입니다.\n여성의 이용건수가 더 높습니다. 여성 이용건수는 전체 가맹점 중 199위(0.51)이고, 남성 이용건수는 전체 가맹점 중 325위(0.49)입니다.\n회원이 가장 많은 나이대는 30대(0.37), 전체 가게 중 93위 입니다. 두 번째로 많은 나이대는 40대(0.24), 전체 가게 중 261위 입니다.\n예약을 하고 방문한 사람들이 더 많고(0.70), 포장배달에 대해서는 매장 문의가 필요합니다.\n대부분 바로 입장(0.99), 10분 이내(0.01)의 대기시간을 가지고 입장합니다. 바로 입장의 대기시간을 가지고 방문한 손님의 비중은 전체 매장 중 3위 이고, 10분 이내의 대기시간을 가지고 방문한 손님의 비중은 전체 매장 중 774위 입니다.\n대부분 여행(0.34), 데이트(0.31) 목적으로 방문합니다. 여행 목적으로 방문한 손님의 비중은 전체 매장 중 1009위 이고, 데이트 목적으로 방문한 손님의 비중은 전체 매장 중 183위 입니다.\n연인・배우자(0.42)와 함께 가장 많이 방문하고, 부모님(0.18)와 함께 두 번째로 많이 방문합니다. 연인・배우자 동반자 유형은 전체 매장 중 393위 이고, 부모님

In [5]:
# '주소' 컬럼에서 '제주시', '서귀포시' 포함 여부에 따라 '주소_시' 컬럼 생성
df['주소_시'] = df['주소'].apply(lambda x: '제주시' if '제주시' in x else ('서귀포시' if '서귀포시' in x else ''))
df['주소_시']

0       서귀포시
1       서귀포시
2       서귀포시
3       서귀포시
4       서귀포시
        ... 
4186    서귀포시
4187    서귀포시
4188     제주시
4189     제주시
4190     제주시
Name: 주소_시, Length: 4191, dtype: object

In [9]:
df['주소_시'].isna().sum()

0

In [10]:
df.columns

Index(['가게명', '주소', '업종', '이용건수구간', '이용금액구간', '건당평균이용금액구간', '월_이용건수비중',
       '화_이용건수비중', '수_이용건수비중', '목_이용건수비중', '금_이용건수비중', '토_이용건수비중', '일_이용건수비중',
       '05-11시_이용건수비중', '12-13시_이용건수비중', '14-17시_이용건수비중', '18-22시_이용건수비중',
       '23-04시_이용건수비중', '현지인_이용건수비중', '남성_이용건수비중', '여성_이용건수비중', '10-20대_회원비중',
       '30대_회원비중', '40대_회원비중', '50대_회원비중', '60대이상_회원비중', '상위키워드', '예약_무',
       '예약_유', '포장배달_유무', '바로 입장', '10분 이내', '30분 이내', '30분 이상', '1시간 이상',
       '2시간 이상', '데이트', '친목', '나들이', '여행', '일상', '기념일', '회식', '가족모임', '비즈니스',
       '연인・배우자', '친구', '지인・동료', '혼자', '아이', '부모님', '반려동물', '친척・형제', '기타',
       '주변관광지', 'mct_요약', 'keyword_요약', '요약', '현지인', '주소_시'],
      dtype='object')

요약 컬럼 임베딩

In [11]:
# 4. 임베딩 모델 로드 (Hugging Face에서 모델 가져오기)
model_name = "jhgan/ko-sroberta-multitask"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [14]:
import chromadb
import os

def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)  # 평균을 내어 문장 임베딩 생성
    return embeddings.squeeze().numpy()

db_folder = r'D:\2024_bigcontest\VectorDB/mct_keyword_v8'
os.makedirs(db_folder, exist_ok=True)  # 폴더가 없을 경우 생성

# ChromaDB 클라이언트 초기화
client = chromadb.PersistentClient(path=db_folder)

# Collection 생성 (이름은 원하는 대로 설정)
collection = client.create_collection("jeju_store_mct_keyword_8")

for index, row in df.iterrows():
    embedding = embed_text(row['요약']).tolist()  # 요약에 대한 임베딩 생성
    collection.add(
        ids=[f'doc_{index}'],  # 고유 ID 설정
        documents=[row['요약']],
        metadatas=[{'name': row['가게명'], 'address': row['주소_시'], 'industry': row['업종'], 'attraction': row['주변관광지'], 'local': row['현지인'], 'summary': row['요약']}],
        embeddings=[embedding]
    )
